# パチスロ末尾予測システム - リファクタリング版

このノートブックは、パッケージ化されたコードを使用する薄いインターフェースです。

## 📋 実行手順
1. セル1: セットアップ（ライブラリインポート）
2. セル2: データ読み込み
3. セル3: 特徴量生成
4. セル4: モデル訓練・予測
5. セル5: 結果表示・可視化

In [ ]:
# ============================================================
# セル1: セットアップ
# ============================================================

import warnings
warnings.filterwarnings('ignore')

# パッケージのインポート
import sys
sys.path.insert(0, '/home/user/last_digit_predictor')

from src import (
    # 設定
    CONFIG, EVENT_DEFINITIONS, check_library_availability,
    # ロガー
    log_info, log_success, log_section,
    # データ読み込み
    load_data_with_events,
    # 特徴量生成
    create_prev_basic_features,
    create_lag_features,
    create_change_features,
    create_auxiliary_features,
    # モデル
    prepare_unified_data,
    select_features_unified,
    run_optuna_optimization,
    make_predictions,
    # 評価
    evaluate_unified_metrics,
    log_unified_metrics,
    # 可視化
    plot_model_performance_comparison,
    plot_feature_importance,
)

# ライブラリの利用可能性をチェック
libs = check_library_availability()
log_info(f"XGBoost: {'✅' if libs['xgboost'] else '❌'}")
log_info(f"LightGBM: {'✅' if libs['lightgbm'] else '❌'}")

log_success("セットアップ完了")

In [ ]:
# ============================================================
# セル2: データ読み込み
# ============================================================

log_section("データ読み込み")

# データベースパスを設定から取得
db_path = CONFIG['DB_PATH']

# データ読み込み（イベントカレンダーと統合）
df_all = load_data_with_events(db_path, table_name='last_digit_summary_all')

log_success(f"データ読み込み完了: {len(df_all)}行")
log_info(f"日付範囲: {df_all['date'].min()} ～ {df_all['date'].max()}")
log_info(f"末尾種類: {df_all['last_digit'].nunique()}種")

In [ ]:
# ============================================================
# セル3: 特徴量生成（元のノートブックと同じ方法）
# ============================================================

log_section("特徴量生成")

import numpy as np

# カラム名を元のノートブックに合わせる
if 'last_digit' in df_all.columns and 'digit_num' not in df_all.columns:
    df_all['digit_num'] = df_all['last_digit']

# データをソート
df_all = df_all.sort_values(['date', 'digit_num']).reset_index(drop=True)

# イベントリストの抽出
event_flags = [col for col in df_all.columns if col.startswith('is_')]
available_events = [col.replace('is_', '') for col in event_flags]

log_info(f"利用可能なイベント: {len(available_events)}個")

# ============================================================
# ラグ対象カラムの自動抽出（元のノートブックと同じロジック）
# ============================================================

exclude_patterns = [
    'date', 'digit_num', 'last_digit', 'machine_', 'is_',
    'prev_', 'allday_', 'distance_', 'match_', 'weekday', 'day_of',
    'max_games', 'min_games'  # リーク防止
]

numeric_cols = df_all.select_dtypes(include=[np.number]).columns.tolist()
lag_target_cols = [
    col for col in numeric_cols
    if not any(pattern in col.lower() for pattern in exclude_patterns)
]

log_info(f"ラグ対象カラム: {len(lag_target_cols)}個")
log_info(f"  例: {lag_target_cols[:5] if len(lag_target_cols) >= 5 else lag_target_cols}")

# パラメータ定義
lag_days = [1, 2, 3, 4, 7, 14, 21, 28]
window_sizes = [1, 2, 3, 4, 7, 14, 21, 28]
change_lags = [1, 7, 14]
stat_windows = [7, 14, 28]

# ============================================================
# 特徴量生成（元のノートブックと同じ順序）
# ============================================================

# 1. ラグ特徴量
log_info("ラグ特徴量を生成中...")
df_all = create_lag_features(df_all, lag_target_cols, lag_days)

# 2. 移動平均・標準偏差
log_info("移動平均・標準偏差を生成中...")
from src.features import create_moving_avg_std_features
df_all = create_moving_avg_std_features(df_all, lag_target_cols, window_sizes)

# 3. 変化量特徴量（正しい引数名: change_lags）
log_info("変化量特徴量を生成中...")
df_all = create_change_features(df_all, lag_target_cols, change_lags)

# 4. ランク変化特徴量
if 'last_digit_rank_diff' in df_all.columns:
    log_info("ランク変化特徴量を生成中...")
    from src.features import create_rank_change_features
    df_all = create_rank_change_features(
        df_all,
        rank_col='last_digit_rank_diff',
        change_lags=change_lags,
        stat_windows=stat_windows
    )

# 5. prev系特徴量（イベント履歴）
log_info("prev系特徴量を生成中...")
metric_cols = [
    'avg_diff_coins', 'avg_games', 'win_rate', 'high_profit_rate',
    'last_digit_rank_diff', 'last_digit_rank_games'
]
# データに存在するカラムのみ使用
metric_cols = [col for col in metric_cols if col in df_all.columns]
from src.features import create_prev_features
df_all = create_prev_features(
    df_all, available_events,
    metric_cols=metric_cols,
    exclude_cols=['max_games', 'min_games']
)

# 6. 補助特徴量
log_info("補助特徴量を生成中...")
df_all = create_auxiliary_features(df_all, available_events)

log_success(f"特徴量生成完了: {df_all.shape[1]}列")

In [ ]:
# ============================================================
# セル4: モデル訓練・予測（TOP1予測の例）
# ============================================================

log_section("モデル訓練・予測")

# 対象イベントを選択
target_event = '1day'  # 1のつく日
task_type = 'binary'   # TOP1予測

log_info(f"対象イベント: {target_event}")
log_info(f"タスク: {task_type}")

# データ準備（訓練/テスト分割）
log_info("データ準備中...")
result = prepare_unified_data(
    df_all,
    event=target_event,
    task_type=task_type,
    rank=1  # TOP1
)

if result is None:
    log_error(f"データ準備失敗: イベント '{target_event}' のデータが不足しています")
else:
    X_train, y_train, X_test, y_test, test_data = result
    
    log_info(f"訓練データ: {X_train.shape[0]}行 × {X_train.shape[1]}列")
    log_info(f"テストデータ: {X_test.shape[0]}行")
    
    # 特徴量選択（任意）
    log_info("特徴量選択中...")
    selected_features = select_features_unified(
        X_train, y_train, X_test,
        task_type=task_type,
        method='ensemble'  # Lasso + F-test + Tree-based
    )
    
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]
    
    log_info(f"選択された特徴量: {len(selected_features)}個")
    
    # モデル訓練（Optuna最適化）
    log_info("モデル訓練中（Optuna最適化）...")
    model_results = run_optuna_optimization(
        X_train_selected, y_train,
        X_test_selected, y_test,
        task_type=task_type,
        n_trials=CONFIG['N_TRIALS']
    )
    
    log_success("モデル訓練完了")
    
    # 評価
    log_info("モデル評価中...")
    best_model = model_results['best_model']
    y_pred = best_model.predict(X_test_selected)
    
    metrics = evaluate_unified_metrics(
        y_test, y_pred, test_data,
        task_type=task_type
    )
    
    log_unified_metrics(metrics, event_name=target_event, task_name='TOP1予測')
    
    # 結果を保存
    results_dict = {
        'event': target_event,
        'task_type': task_type,
        'model': best_model,
        'selected_features': selected_features,
        'metrics': metrics,
        'y_pred': y_pred,
        'y_test': y_test,
        'test_data': test_data
    }

In [ ]:
# ============================================================
# セル5: 結果可視化
# ============================================================

log_section("結果可視化")

# 特徴量重要度（Random Forestの場合）
if hasattr(results_dict['model'], 'feature_importances_'):
    log_info("特徴量重要度を表示中...")
    plot_feature_importance(
        results_dict['model'],
        results_dict['selected_features'],
        top_n=20
    )

log_success("可視化完了")

## 📊 まとめ

このノートブックでは、リファクタリングされたパッケージを使用して：
1. データの読み込み
2. 特徴量生成
3. モデル訓練・最適化
4. 評価・可視化

を実行しました。

### コードの改善点
- ✅ print文 → loggingモジュールに置き換え
- ✅ 重複関数の削除
- ✅ 機能別にモジュール分割
- ✅ ノートブックが薄く、可読性向上

### 次のステップ
- 他のイベント（4day, 0day, 40day）でも同様に実行
- TOP2予測、ランク予測にも対応
- 複数モデルのアンサンブル予測